In [7]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
import os.path
import numpy as np

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.types import StructType, StructField, DoubleType
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [8]:
# Paths
data_path = "/Users/alek/Downloads/H3/MMDS_H3/data"
dataset_path = os.path.join(data_path, "dataset/diabetes_binary_health_indicators_BRFSS2015.csv")
trained_model_path = os.path.join(data_path, "trained_models")

offline_path = os.path.join(trained_model_path, "offline.csv")
online_path = os.path.join(trained_model_path, "online.csv")


## Create 80/20 random split

In [17]:
df = pd.read_csv(dataset_path)

# Diabetes_binary is the label column
y = df["Diabetes_binary"]  
X = df.drop("Diabetes_binary", axis=1)

X_offline, X_online, y_offline, y_online = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Combine X_offline + y_offline and X_online + y_online
offline = pd.concat([X_offline, y_offline], axis=1)
online = pd.concat([X_online, y_online], axis=1)

offline.to_csv(offline_path, index=False)
online.to_csv(online_path, index=False)

## Offline Spark functionality

In [10]:
spark = SparkSession.builder \
    .appName("OfflineModelTraining") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .getOrCreate()

schema = StructType([
    StructField("HighBP", DoubleType(), True),
    StructField("HighChol", DoubleType(), True),
    StructField("CholCheck", DoubleType(), True),
    StructField("BMI", DoubleType(), True),
    StructField("Smoker", DoubleType(), True),
    StructField("Stroke", DoubleType(), True),
    StructField("HeartDiseaseorAttack", DoubleType(), True),
    StructField("PhysActivity", DoubleType(), True),
    StructField("Fruits", DoubleType(), True),
    StructField("Veggies", DoubleType(), True),
    StructField("HvyAlcoholConsump", DoubleType(), True),
    StructField("AnyHealthcare", DoubleType(), True),
    StructField("NoDocbcCost", DoubleType(), True),
    StructField("GenHlth", DoubleType(), True),
    StructField("MentHlth", DoubleType(), True),
    StructField("PhysHlth", DoubleType(), True),
    StructField("DiffWalk", DoubleType(), True),
    StructField("Sex", DoubleType(), True),
    StructField("Age", DoubleType(), True),
    StructField("Education", DoubleType(), True),
    StructField("Income", DoubleType(), True),
    StructField("Diabetes_binary", DoubleType(), True)
])

df_offline = spark.read.csv(offline_path, header=True, schema=schema) # we're inferring the schema, but we can create one as well

label_column = "Diabetes_binary"
feature_columns = [c for c in df_offline.columns if c != label_column]
df_offline = df_offline.withColumnRenamed(label_column, "label")

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_assembled") # assembles feature vector

scaler = StandardScaler(inputCol="features_assembled", outputCol="features", withStd = True, withMean = False)

# Classifiers
lr = LogisticRegression(featuresCol='features', labelCol="label")
rf = RandomForestClassifier(featuresCol='features', labelCol="label")
gbt = GBTClassifier(featuresCol='features', labelCol="label")

pipeline_lr = Pipeline(stages=[assembler, scaler, lr])
pipeline_rf = Pipeline(stages=[assembler, scaler, rf])
pipeline_gbt = Pipeline(stages=[assembler, scaler, gbt])

paramGridLR = (ParamGridBuilder()
    .addGrid(lr.regParam, [0.01, 0.1, 0.5])
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
    .build()
)

paramGridRF = (ParamGridBuilder()
    .addGrid(rf.numTrees, [10, 20, 50])
    .addGrid(rf.maxDepth, [5, 10])
    .addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt', 'log2'])
    .build()
)

paramGridGBT = (ParamGridBuilder()
    .addGrid(gbt.maxIter, [10, 20])
    .addGrid(gbt.maxDepth, [3, 5])
    .build()
)


evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="f1")

cv_lr = CrossValidator(
    estimator=pipeline_lr,
    estimatorParamMaps=paramGridLR,
    evaluator=evaluator,
    numFolds=3
)

cv_rf = CrossValidator(
    estimator=pipeline_rf,
    estimatorParamMaps=paramGridRF,
    evaluator=evaluator,
    numFolds=3
)

cv_gbt = CrossValidator(
    estimator=pipeline_gbt,
    estimatorParamMaps=paramGridGBT,
    evaluator=evaluator,
    numFolds=3
)

In [11]:
cvModel_lr = cv_lr.fit(df_offline)   # fits pipeline_lr with paramGridLR

25/02/13 18:33:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/02/13 18:33:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [12]:
cvModel_rf = cv_rf.fit(df_offline)   # fits pipeline_rf with paramGridRF

25/02/13 18:34:04 WARN DAGScheduler: Broadcasting large task binary with size 1326.0 KiB
25/02/13 18:34:07 WARN DAGScheduler: Broadcasting large task binary with size 1326.0 KiB
25/02/13 18:34:10 WARN DAGScheduler: Broadcasting large task binary with size 1326.0 KiB
25/02/13 18:34:19 WARN DAGScheduler: Broadcasting large task binary with size 1449.1 KiB
25/02/13 18:34:20 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
25/02/13 18:34:21 WARN DAGScheduler: Broadcasting large task binary with size 1288.2 KiB
25/02/13 18:34:24 WARN DAGScheduler: Broadcasting large task binary with size 1449.1 KiB
25/02/13 18:34:25 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
25/02/13 18:34:26 WARN DAGScheduler: Broadcasting large task binary with size 1288.2 KiB
25/02/13 18:34:30 WARN DAGScheduler: Broadcasting large task binary with size 1449.1 KiB
25/02/13 18:34:30 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
25/02/13 18:34:31 WARN DAGSche

In [13]:
cvModel_gbt = cv_gbt.fit(df_offline) # fits pipeline_gbt with paramGridGBT

In [14]:
bestF1_lr = max(cvModel_lr.avgMetrics)  # best F1 from logistic regression grid search
bestF1_rf = max(cvModel_rf.avgMetrics)  # best F1 from random forest grid search
bestF1_gbt = max(cvModel_gbt.avgMetrics) # best F1 from GBT grid search

print("LR best F1: ", bestF1_lr)
print("RF best F1: ", bestF1_rf)
print("GBT best F1:", bestF1_gbt)

# Determine overall best
if bestF1_lr >= bestF1_rf and bestF1_lr >= bestF1_gbt:
    best_overall_model = cvModel_lr.bestModel
    best_score = bestF1_lr
    best_model_name = "LogisticRegression"
elif bestF1_rf >= bestF1_lr and bestF1_rf >= bestF1_gbt:
    best_overall_model = cvModel_rf.bestModel
    best_score = bestF1_rf
    best_model_name = "RandomForest"
else:
    best_overall_model = cvModel_gbt.bestModel
    best_score = bestF1_gbt
    best_model_name = "GBT"

print(f"Overall best model is {best_model_name} with F1 = {best_score}")

LR best F1:  0.826976776194452
RF best F1:  0.8222871575772458
GBT best F1: 0.8296026843595836
Overall best model is GBT with F1 = 0.8296026843595836


In [15]:
best_overall_model.write().overwrite().save(trained_model_path)
print(f"Saved best model ({best_model_name}) to {trained_model_path}")

Saved best model (GBT) to /Users/alek/Downloads/H3/MMDS_H3/data/trained_models


In [16]:
spark.stop()